<a href="https://colab.research.google.com/github/Hlamulo1/Federated-research/blob/main/federated_approach.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_federated as tff

# Load and preprocess the dataset
from google.colab import drive
drive.mount('/content/drive')

%cd '/content/drive/My Drive/MalwareData'

import pandas as pd

file_path = 'MalwareData.csv'

malData = pd.read_csv(file_path, sep='|', low_memory=True)

malData = malData.drop(['Name', 'md5'], axis=1)

# Split the dataset into legitimate and malware subsets
legit = malData[0:41323].drop(["legitimate"], axis=1)
mal = malData[41323::].drop(["legitimate"], axis=1)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(malData.drop(['legitimate'], axis=1), malData['legitimate'], test_size=0.2, random_state=42)

# Define a function to create a federated dataset
def create_federated_data(X, y):
    dataset = tf.data.Dataset.from_tensor_slices((X.values, y.values.astype(np.int32)))
    return [dataset]  # Wrap the dataset in a list for TFF

train_data = create_federated_data(X_train, y_train)

# Define a TFF model function
def create_federated_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(54,)),
        tf.keras.layers.Dense(16, activation='relu'),
        tf.keras.layers.Dense(8, activation='relu'),
        tf.keras.layers.Dense(4, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    return tff.learning.from_keras_model(model, loss=tf.keras.losses.BinaryCrossentropy(), metrics=[tf.keras.metrics.BinaryAccuracy()])

# Federated Learning setup
tff.backends.native.set_local_python_execution_context()

federated_train_data = [create_federated_data(X, y) for X, y in zip(np.array_split(X_train, 10), np.array_split(y_train, 10))]

train_data_type = tff.to_type(train_data[0].element_spec)

@tff.tf_computation(train_data_type)
def initialize_fn(train_data):
    model = create_federated_model()
    return tff.learning.from_keras_model(model, loss=tf.keras.losses.BinaryCrossentropy(), metrics=[tf.keras.metrics.BinaryAccuracy()])

@tff.federated_computation
def next_fn(server_weights, federated_dataset):
    model = create_federated_model()
    server_weights.assign_weights_to(model)
    return tff.learning.build_federated_averaging_process(model, server_optimizer_fn=lambda: tf.keras.optimizers.Adam(learning_rate=0.01))

iterative_process = tff.templates.IterativeProcess(initialize_fn=initialize_fn, next_fn=next_fn)

state = iterative_process.initialize(train_data[0])

NUM_ROUNDS = 10
for round_num in range(NUM_ROUNDS):
    state, metrics = iterative_process.next(state, federated_train_data)
    print(f'Round {round_num}: {metrics}')

# Federated evaluation
def federated_evaluate(model, test_data):
    test_metrics = tff.learning.build_federated_evaluation(model)
    return test_metrics(test_data)

federated_model = create_federated_model()
federated_eval_metrics = federated_evaluate(federated_model, train_data)
print(f'Federated Evaluation: {federated_eval_metrics}')

# The federated model can be used for predictions


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/MalwareData


AttributeError: ignored

In [12]:
import tensorflow as tf
import tensorflow_federated as tff

# Check the version of TFF (you can install the latest version if needed)
print(f'TensorFlow Federated version: {tff.__version__}')

# Set the runtime to eager execution
tf.compat.v1.enable_eager_execution()

# Check that TensorFlow is running in eager mode
print(f'TensorFlow eager execution: {tf.executing_eagerly()}')

# Create a reference to a local executor (e.g., for a single machine)
local_executor = tff.framework.EagerTFExecutor()

# Set the default context to use the local executor
tff.framework.set_default_context(tff.framework.ExecutionContext(executor=local_executor))


TensorFlow Federated version: 0.61.0
TensorFlow eager execution: True


AttributeError: ignored

In [10]:
import tensorflow_federated as tff

# Create a local executor
local_executor = tff.framework.LocalExecutor()

# Set the default context to use the local executor
tff.framework.set_default_context(
    tff.framework.ExecutionContext(executor_fn=lambda: local_executor))


AttributeError: ignored

In [5]:
!pip install --upgrade tensorflow-federated


In [10]:
!pip install --upgrade tensorflow


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 524.1/524.1 MB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 37.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 96.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.8/440.8 kB 44.3 MB/s eta 0:00:00
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.12.0
    Uninstalling tensorflow-estimator-2.12.0:
      Successfully uninstalled tensorflow-estimator-2.12.0
  Attempting uninstall: keras
    Found existing installation: keras 2.12.0
    Uninstalling keras-2.12.0:
      Successfully uninstalled keras-2.12.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.12.3
    Uninstalling tensorboard-2.12.3:
      Successfully uninstalled tensorboard-2.12.3
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.12.0
    Uninstalling tensorflow-2.12.0:
      Successfully un

In [5]:
from sklearn.model_selection import train_test_split

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(malData.drop(['legitimate'], axis=1), malData['legitimate'], test_size=0.2, random_state=42)


In [3]:
!pip install --upgrade tensorflow-federated


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 585.9/585.9 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 92.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 95.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.7/440.7 kB 46.5 MB/s eta 0:00:00
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.13.0
    Uninstalling tensorflow-estimator-2.13.0:
      Successfully uninstalled tensorflow-estimator-2.13.0
  Attempting uninstall: keras
    Found existing installation: keras 2.13.1
    Uninstalling keras-2.13.1:
      Successfully uninstalled keras-2.13.1
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.13.0
    Uninstalling tensorboard-2.13.0:
      Successfully uninstalled tensorboard-2.13.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.13.0
    Uninstalling tensorflow-2.13.0:
      Successfully un

In [1]:
import tensorflow_federated as tff
